# PID Control Introduction

Introduces PID control with a simple 1D quadrotor example.

$u(t) = K_p e(t) + K_i \int_0^t e(t) dt + K_d \dfrac{de(t)}{dt}$

$\dfrac{U(s)}{E(s)} = K_p + K_i \frac{1}{s} + K_d s$

In [1]:
import scipy.integrate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [11]:
class PID:

    def __init__(self, Kp, Ki, Kd):
        self.e_i = 0
        self.e_last = None
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd

    def control(self, t, e, dt):
        if self.e_last is None:
            self.e_last = e
        self.e_i = saturate(self.e_i + e*dt, -1, 1)  # error integral, using euler integration and windup guard
        e_d = (e - self.e_last)/dt  # error derivative using finite difference approx
        self.e_last = e        
        return self.Kp*e + self.Ki*self.e_i + self.Kd*e_d

def saturate(u, u_min, u_max):
    if u < u_min:
        return u_min
    elif u > u_max:
        return u_max
    else:
        return u

In [12]:
def quad_1d_dyn(t, state, inputs, params):
    m, g= params
    thrust_d = inputs  # desired thrust
    z, vz, thrust = state
    dthrust = 20*(thrust_d - thrust)  # motor lag
    Fz = thrust - m*g
    az = Fz/m
    return (vz, az, dthrust)

In [13]:
def simulate(controller, tf=10, dt=0.01):
    t = np.arange(0, tf, dt)
    x0 = [0, 0, 0]
    m = 1.1 # mass of drone
    g = 9.8  # accel of gravity
    params = [m, g]
    m_trim = 1 # mass assumed for trim
    trim_thrust = m_trim*g
    r = 2  # desired altitude
    x = []
    u = []
    for ti in t:
        thrust = saturate(controller.control(t, r - x0[0], dt) + trim_thrust, 0, 20)
        res = scipy.integrate.solve_ivp(fun=lambda t, state: quad_1d_dyn(t, state, thrust, params),
                                  t_span=[ti, ti + dt], y0=x0, t_eval=[ti + dt])
        x0 = list(res['y'].reshape(-1))
        x.append(x0)
        u.append([thrust])
    x = np.array(x)  # convert to 2D np array
    u = np.array(u)
    return  {
        'dt': dt,
        't': t,
        'x': x,
        'u': u
    }

In [14]:
def drone_1d_anim(data, dt_anim=0.15):
    t = data['t']
    xdata, ydata = [], []
    step = int(np.floor(dt_anim/data['dt']))
    
    im = plt.imread('./drone.png')
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

    ax1.set_xlim(-2, 2)
    ax1.set_ylim(0, 4)
    ax1.axis('off')
    ax1.hlines(2, -2, 2, linestyles='dashed', alpha=0.5, label='reference')
    ax1.hlines(0.05, -2, 2, colors='g', alpha=1, linewidth=5, label='ground')
    ax1.legend()
    ln1, = ax1.plot([], [], 'ro', zorder=2, label='vehicle')

    newax = fig.add_axes([0.19, 0, 0.1, 0.1], zorder=1)
    newax.imshow(im, alpha=0.5)
    newax.axis('off')

    ax2.set_title('Altitude')
    ax2.plot(t, data['x'][:, 0])
    ln2, = ax2.plot([], [], 'ro', zorder=2, label='vehicle')
    ax2.hlines(2, t[0], t[-1], linestyles='dashed', alpha=0.5, label='reference')
    ax2.hlines(0, t[0], t[-1], colors='g', alpha=1, linewidth=5, label='ground')
    ax2.set_xlabel('t, sec')
    ax2.set_ylabel('z, m')
    ax2.grid(True)
    ax2.legend()

    ax3.set_title('Thrust')
    ax3.plot(t, data['u'][:, 0], label='command')
    ax3.plot(t, data['x'][:, 2], label='actual')
    ln3, = ax3.plot([], [], 'rx')
    ln4, = ax3.plot([], [], 'ro')
    ax3.set_xlabel('t, sec')
    ax3.set_ylabel('thrust, N')
    ax3.grid(True)
    ax3.legend()

    def update(frame):
        xdata = [0]
        z = data['x'][step*frame, 0]
        thrust = data['x'][step*frame, 2]
        u = data['u'][step*frame, 0]
        ydata = [z]
        newax.set_position([0.19, z/(5.3) + 0.06, 0.1, 0.1], which='both')
        ln1.set_data(xdata, ydata)
        ln2.set_data(data['t'][step*frame], z)
        ln3.set_data(data['t'][step*frame], u)
        ln4.set_data(data['t'][step*frame], thrust)
        return ln1, ln2
    ani = FuncAnimation(fig, update, frames=range(len(t)//step), interval=1000*dt_anim)
    plt.close()
    return HTML(ani.to_html5_video())

## Simulation

### Case 1

In [15]:
data = simulate(controller=PID(Kp=3, Ki=0, Kd=0), tf=5)
drone_1d_anim(data)

This first case is just using proportional gain, and is undamped, so the oscillation continues with the same magnitude.

### Case 2

In [16]:
data = simulate(controller=PID(Kp=10, Ki=0, Kd=5), tf=5)
drone_1d_anim(data)

In this case, the added derivative gain help stop oscillations and minimizes overshoot.

### Case 3

In [17]:
data = simulate(controller=PID(Kp=10, Ki=1, Kd=5), tf=5)
drone_1d_anim(data)

Increasing the integral reduces the steady state error, but note to get the fast response we need to saturate the error integral using a wind-up guard.